# 10.2 NGSolve with tensor-flow

M. Feischl + J. Schöberl 

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

fes = H1(mesh, order=2, dirichlet=".*")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)
f = LinearForm(v*dx)
gfu = GridFunction(fes)

deform = GridFunction(VectorH1(mesh, order=1))

solve a parametric problem:

In [ ]:
def Solve(Ax, Ay):
    mesh.UnsetDeformation()
    deform.Interpolate ( (x*y*Ax, x*y*Ay) )
    mesh.SetDeformation(deform)
    a.Assemble()
    f.Assemble()
    gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

In [ ]:
Solve(0.8, 0.5)
Draw (gfu);

In [ ]:
# np.asarray (gfu.vec)

In [ ]:
n_data = 50   # number of datapoints
input_dim = 2   # dimension of each datapoint
data_in = np.random.uniform(0,1,size=(n_data,input_dim))  # artificial datapoints
# print (data_in)

In [ ]:
output_dim = fes.ndof
data_out = np.zeros((n_data, output_dim))

for i, (ax,ay) in enumerate(data_in):
    Solve (ax, ay)
    data_out[i,:] = gfu.vec

In [ ]:
print (output_dim)

start the training ...

In [ ]:
import tensorflow as tf

# func = 'relu' #activation function
func = 'swish' #activation function
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10,input_shape=(input_dim,),activation=func),
    tf.keras.layers.Dense(20,activation=func),
    tf.keras.layers.Dense(50,activation=func),
    tf.keras.layers.Dense(output_dim)
    ])

# Standard Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Loss function (error functional) mean squared error (least squares)
@tf.function
def loss_fn(y_true, y_pred):
   return tf.math.reduce_mean(tf.math.square(y_true-y_pred))

loss_fn = tf.keras.losses.MeanSquaredError()

# Training
epochs = 1000 # number of times the data is used for training
batch_size = 1024  # each gradient descent step uses 1024 datapoints
model.compile(optimizer=optimizer,loss=loss_fn)
model.fit(data_in,data_out,epochs=epochs,batch_size=batch_size,verbose=0)
print(loss_fn(data_out,model(data_in)).numpy())

In [ ]:
output = model(np.array([[0.1,0.2]]))
outputvec = output.numpy().flatten()

Solve (0.1, 0.2)
Draw (gfu)

gfumodel = GridFunction(fes)
gfumodel.vec.FV()[:] = outputvec
    
Draw (gfumodel)
Draw (gfu-gfumodel, mesh);
print ("err = ", Integrate((gfu-gfumodel)**2, mesh)**0.5)